## Data Description

You are provided with 25,010 poker hands in train.csv and 1,000,000 in test.csv. Each hand consists of five cards with a given suit and rank, drawn from a standard deck of 52. Suits and ranks are represented as ordinal categories:

```
S1 “Suit of card #1”
Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}
C1 “Rank of card #1”
Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King)

...

S5 “Suit of card #5”
C5 “Rank of card #5”
```

Each row in the training set has the accompanying class label for the poker hand it comprises. The hands are omitted from the test set and must be predicted by participants. Hands are classified into the following ordinal categories:


```
0: Nothing in hand; not a recognized poker hand 
1: One pair; one pair of equal ranks within five cards
2: Two pairs; two pairs of equal ranks within five cards
3: Three of a kind; three equal ranks within five cards
4: Straight; five cards, sequentially ranked with no gaps
5: Flush; five cards with the same suit
6: Full house; pair + different rank three of a kind
7: Four of a kind; four equal ranks within five cards
8: Straight flush; straight + flush
9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush
```
Note that the Straight flush and Royal flush hands are not representative of
the true domain because they have been over-sampled. The straight flush
is 14.43 times more likely to occur in the training set, while the royal flush is 129.82 times more likely.

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from functools import reduce

## Train data

In [3]:
suits = ['S1','S2','S3','S4','S5']
cards = ['C1','C2','C3','C4','C5']
distances = ['D1', 'D2', 'D3', 'D4', 'D5']

In [4]:
train_dataset = pd.read_csv('train.csv', delimiter=',')

In [5]:
train_dataset.head()

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,hand
0,4,9,2,1,2,2,4,7,2,8,0
1,1,4,3,6,1,12,3,11,2,7,0
2,1,11,4,1,3,7,4,11,2,1,2
3,2,9,2,4,3,6,1,9,4,9,3
4,1,8,2,4,2,11,2,2,2,1,0


In [6]:
data = train_dataset.drop('hand', axis=1)
train_labels = train_dataset['hand']

In [7]:
labels_vect = tf.keras.utils.to_categorical(train_labels)

#### Adding distance features

In [8]:
def get_distances(hand):
    distances_list = []
    for i, card in enumerate(hand):
        if (i < len(hand) - 1):
            distances_list.append(hand[i+1] - card)
        else:
            distances_list.append(13 - card + hand[0])
    return pd.Series(distances_list)

def with_distances(data):
    sorted = data[cards]
    sorted.values.sort()

    distances_df = sorted.apply(get_distances, axis=1)
    distances_df.columns = distances
    
    data_with_distances = pd.concat([data, distances_df], axis=1)
    return data_with_distances

In [10]:
data_with_distances = with_distances(data)
data_with_distances.head()

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,D1,D2,D3,D4,D5
0,4,9,2,1,2,2,4,7,2,8,1,5,1,1,5
1,1,4,3,6,1,12,3,11,2,7,2,1,4,1,5
2,1,11,4,1,3,7,4,11,2,1,0,6,4,0,3
3,2,9,2,4,3,6,1,9,4,9,2,3,0,0,8
4,1,8,2,4,2,11,2,2,2,1,1,2,4,3,3


#### One hot encoding

In [11]:
def one_hot_encoded(data, columns, encoder, mode='transform'):
    encoded = None
    if (mode == 'fit'):
        encoded = encoder.fit_transform(data[columns])
    elif (mode == 'transform'):
        encoded = encoder.transform(data[columns])
    return pd.concat([data.drop(columns, axis=1), pd.DataFrame(encoded.toarray())], axis=1)

In [12]:
encoder = OneHotEncoder()
data_encoded = one_hot_encoded(data_with_distances, suits, encoder, 'fit')
print(data_encoded.shape)

data_encoded.head()

(25010, 30)


,C1,C2,C3,C4,C5,D1,D2,D3,D4,D5,...,10,11,12,13,14,15,16,17,18,19
0,9,1,2,7,8,1,5,1,1,5,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,4,6,12,11,7,2,1,4,1,5,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,11,1,7,11,1,0,6,4,0,3,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,9,4,6,9,9,2,3,0,0,8,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,8,4,11,2,1,1,2,4,3,3,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


#### Normalization

In [13]:
def normalized(data, columns, mean, std):
    data_normalized = (data[columns] - mean) / std
    to_return = pd.concat([data_normalized, data.drop(columns, axis=1)], axis=1)
    return to_return

In [14]:
mean_c = data_encoded[cards].mean().mean()
std_c = pd.Series(data_encoded[cards].values.flatten()).std()
mean_d = data_encoded[distances].mean().mean()
std_d = pd.Series(data_encoded[distances].values.flatten()).std()

data_normalized = normalized(data_encoded, cards, mean_c, std_c)
data_normalized = normalized(data_normalized, distances, mean_d, std_d)
data_normalized.head()

,D1,D2,D3,D4,D5,C1,C2,C3,C4,C5,...,10,11,12,13,14,15,16,17,18,19
0,-0.775959,1.163939,-0.775959,-0.775959,1.163939,0.537109,-1.596136,-1.329480,0.003798,0.270453,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,-0.290985,-0.775959,0.678965,-0.775959,1.163939,-0.796169,-0.262858,1.337076,1.070420,0.003798,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,-1.260934,1.648914,0.678965,-1.260934,0.193990,1.070420,-1.596136,0.003798,1.070420,-1.596136,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,-0.290985,0.193990,-1.260934,-1.260934,2.618863,0.537109,-0.796169,-0.262858,0.537109,0.537109,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-0.775959,-0.290985,0.678965,0.193990,0.193990,0.270453,-0.796169,1.070420,-1.329480,-1.596136,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [15]:
data_normalized.shape

(25010, 30)

#### Test function

In [16]:
def calc_hand_label(hand):    
    def f(hand):
        suits_hist = reduce(lambda d,x: {**d, **{x: (d.get(x, 0) + 1)}}, hand[:,0], {})
        ranks_hist = reduce(lambda d,x: {**d, **{x: (d.get(x, 0) + 1)}}, hand[:,1], {})
        
        if len(ranks_hist.values()) < 5:
            if len(ranks_hist.values()) == 2:
                if max(list(ranks_hist.values())) == 4:
                    return 7
    #                 print('7: Four of a kind; four equal ranks within five cards')
                else:
                    return 6
    #                 print('6 Full house; pair + different rank three of a kind')
            elif len(ranks_hist.values()) == 3:
                if max(list(ranks_hist.values())) == 3:
                    return 3
    #                 print('3: Three of a kind; three equal ranks within five cards')
                else:
                    return 2
    #                 print('2: Two pairs; two pairs of equal ranks within five cards')
            else:
                return 1
    #             print('1: One pair; one pair of equal ranks within five cards')
        else:
            if len(suits_hist.values()) == 1:
                if max(list(ranks_hist.keys())) -  min(list(ranks_hist.keys())) == 4:
                    if max(list(ranks_hist.keys())) == 13:
                        return 9
    #                     print('9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush')
                    else:
                        return 8
    #                     print('8: Straight flush; straight + flush')
                else:
                    return 5
    #                 print('5: Flush; five cards with the same suit')
            elif max(list(ranks_hist.keys())) -  min(list(ranks_hist.keys())) == 4:
                return 4    
    #             print('4: Straight; five cards, sequentially ranked with no gaps')
            else:
                return 0
                print('0: Nothing in hand; not a recognized poker hand ')
                
    hand = np.array(hand).reshape(5,2)
    res1 = f(hand)
    
    hand[:,1] = list(map(lambda x: 13 if (x == 1) else x-1, hand[:,1]))
    res2 = f(hand)
    
    if res1 == 9 and res2 == 8:
        return res2
    else:
        return max(res1, res2)

#### Encode cards

In [17]:
def card_encode(s, c):
    cards=["A","2","3","4","5","6","7","8","9","10","J","Q","K"]
    suits="♥♦♣♠"
    return cards[c - 1] + suits[s - 1]

def hand_encode(hand):
    return ', '.join(map(lambda x: card_encode(hand[x*2], hand[x*2+1]),range(5)))

print(hand_encode([4,9,2,1,2,2,4,7,2,8]))
print(hand_encode([2,9,2,4,3,6,1,9,4,9]))

9♠, A♦, 2♦, 7♠, 8♦
9♦, 4♦, 6♣, 9♥, 9♠


## Neural Network

In [18]:
from time import time
from keras.callbacks import TensorBoard

model = keras.Sequential()
#input
model.add(keras.layers.Dense(120, activation='relu', input_shape=(30,)))
model.add(keras.layers.Dropout(0.2))
#hidden
model.add(keras.layers.Dense(240, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(120, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(60, activation='relu'))
model.add(keras.layers.Dropout(0.2))
# Add a softmax layer with 10 output units:
model.add(keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer=tf.train.AdamOptimizer(0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

model.fit(np.array(data_normalized), np.array(labels_vect), epochs=500, batch_size=512)

Using TensorFlow backend.


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 34us/step - loss: 1.3139 - acc: 0.4383

Epoch 2/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 23us/step - loss: 1.0426 - acc: 0.5071

Epoch 3/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 22us/step - loss: 0.9901 - acc: 0.549925010 [========>.....................] - ETA: 0s - loss: 0.9956 - acc:

Epoch 4/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 25us/step - loss: 0.9347 - acc: 0.5867

Epoch 5/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 24us/step - loss: 0.8543 - acc: 0.6349

Epoch 6/500
25010/25010 [=============================

25010/25010 [==============================]25010/25010 [==============================] - 1s 26us/step - loss: 0.0185 - acc: 0.9954

Epoch 52/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 23us/step - loss: 0.0176 - acc: 0.9957

Epoch 53/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 25us/step - loss: 0.0166 - acc: 0.9957

Epoch 54/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 24us/step - loss: 0.0181 - acc: 0.9955

Epoch 55/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 29us/step - loss: 0.0159 - acc: 0.9960

Epoch 56/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 28us/step - loss: 0.0158 - acc: 0.9961

Epoch 57/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 25us/step - loss: 0.0167

25010/25010 [==============================]25010/25010 [==============================] - 1s 24us/step - loss: 0.0048 - acc: 0.9986

Epoch 107/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 24us/step - loss: 0.0049 - acc: 0.9985

Epoch 108/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 23us/step - loss: 0.0046 - acc: 0.9988

Epoch 109/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 22us/step - loss: 0.0039 - acc: 0.9988

Epoch 110/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 23us/step - loss: 0.0043 - acc: 0.9988

Epoch 111/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 25us/step - loss: 0.0048 - acc: 0.9986

Epoch 112/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 24us/step - loss: 

25010/25010 [==============================]25010/25010 [==============================] - 1s 30us/step - loss: 0.0018 - acc: 0.9994

Epoch 162/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 30us/step - loss: 0.0020 - acc: 0.9994

Epoch 163/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 30us/step - loss: 0.0024 - acc: 0.9993

Epoch 164/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 30us/step - loss: 0.0021 - acc: 0.9995

Epoch 165/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 29us/step - loss: 0.0020 - acc: 0.9996

Epoch 166/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 29us/step - loss: 0.0017 - acc: 0.9997

Epoch 167/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 29us/step - loss: 

...

25010/25010 [==============================]25010/25010 [==============================] - 1s 29us/step - loss: 7.2685e-04 - acc: 0.9998

Epoch 271/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 29us/step - loss: 0.0010 - acc: 0.9997

Epoch 272/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 38us/step - loss: 0.0015 - acc: 0.9994

Epoch 273/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 36us/step - loss: 8.2989e-04 - acc: 0.9998

Epoch 274/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 37us/step - loss: 0.0015 - acc: 0.9998

Epoch 275/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 35us/step - loss: 0.0022 - acc: 0.9995

Epoch 276/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 37us/step 

25010/25010 [==============================]25010/25010 [==============================] - 1s 37us/step - loss: 0.0018 - acc: 0.9995

Epoch 325/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 33us/step - loss: 0.0017 - acc: 0.9996

Epoch 326/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 35us/step - loss: 0.0021 - acc: 0.9996

Epoch 327/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 47us/step - loss: 0.0014 - acc: 0.9996

Epoch 328/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 34us/step - loss: 0.0010 - acc: 0.9997

Epoch 329/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 32us/step - loss: 0.0012 - acc: 0.9997

Epoch 330/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 37us/step - loss: 

25010/25010 [==============================]25010/25010 [==============================] - 1s 38us/step - loss: 5.4679e-04 - acc: 0.9998

Epoch 379/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 31us/step - loss: 7.7288e-04 - acc: 0.9999

Epoch 380/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 44us/step - loss: 0.0012 - acc: 0.9998

Epoch 381/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 44us/step - loss: 0.0015 - acc: 0.9995

Epoch 382/500
25010/25010 [==============================]25010/25010 [==============================] - 2s 71us/step - loss: 9.2124e-04 - acc: 0.9997

Epoch 383/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 50us/step - loss: 0.0016 - acc: 0.9997

Epoch 384/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 26us/s

25010/25010 [==============================]25010/25010 [==============================] - 1s 51us/step - loss: 5.0753e-04 - acc: 0.9999

Epoch 433/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 57us/step - loss: 6.5982e-04 - acc: 0.9999

Epoch 434/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 51us/step - loss: 0.0014 - acc: 0.9996

Epoch 435/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 48us/step - loss: 9.8920e-04 - acc: 0.9998

Epoch 436/500
25010/25010 [==============================]25010/25010 [==============================] - 2s 71us/step - loss: 4.2207e-04 - acc: 0.9999

Epoch 437/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 43us/step - loss: 8.5744e-04 - acc: 0.9997

Epoch 438/500
25010/25010 [==============================]25010/25010 [==============================] - 4

25010/25010 [==============================]25010/25010 [==============================] - 1s 28us/step - loss: 2.0549e-04 - acc: 1.0000

Epoch 487/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 29us/step - loss: 5.7528e-04 - acc: 0.9998

Epoch 488/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 29us/step - loss: 3.1445e-04 - acc: 0.9999

Epoch 489/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 31us/step - loss: 3.6283e-04 - acc: 0.9998

Epoch 490/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 29us/step - loss: 0.0012 - acc: 0.9998

Epoch 491/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 29us/step - loss: 0.0016 - acc: 0.9996

Epoch 492/500
25010/25010 [==============================]25010/25010 [==============================] - 1s 31

## Test data

In [19]:
test_dataset = pd.read_csv('test.csv', delimiter=',')
test_data = test_dataset.drop('id', axis=1)

In [20]:
test_labels = test_data.apply(calc_hand_label, axis=1)

In [21]:
test_data_with_distances = with_distances(test_data)
test_data_encoded = one_hot_encoded(test_data_with_distances, suits, encoder)
test_data_normalized = normalized(test_data_encoded, cards, mean_c, std_c)
test_data_normalized = normalized(test_data_normalized, distances, mean_d, std_d)

In [22]:
test_data_normalized.shape

(1000000, 30)

## Predictions

In [23]:
test_data_sample_predicted_10 = model.predict(np.array(test_data_normalized))
test_data_sample_predicted = np.array(list(map(np.argmax, test_data_sample_predicted_10)))

In [24]:
predictions_data = pd.concat([
    test_dataset,
    pd.DataFrame({'predictions': test_data_sample_predicted}),
    pd.DataFrame({'labels': test_labels}),
], axis=1)

In [25]:
wrong_answers = predictions_data[predictions_data['predictions'] != predictions_data['labels']]
wrong_answers.shape

(247, 13)

In [26]:
wrong_answers['labels'].value_counts()

4    208
0     20
5     19
Name: labels, dtype: int64

In [27]:
wrong_answers[0:10]

,id,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,predictions,labels
4202,4203,4,10,2,11,1,12,3,13,4,1,0,4
6823,6824,1,10,3,1,3,11,1,13,4,12,0,4
18740,18741,1,11,1,1,3,13,3,10,2,12,0,4
20512,20513,2,11,4,12,1,13,3,1,1,10,0,4
24421,24422,1,4,1,2,4,3,4,1,1,5,0,4
28433,28434,4,5,3,7,3,4,3,8,3,6,8,4
29282,29283,4,12,1,1,4,13,4,11,3,10,0,4
36911,36912,2,13,3,11,3,1,2,10,1,12,0,4
41314,41315,1,13,1,12,1,1,3,10,3,11,0,4
46211,46212,3,10,3,8,3,13,3,9,3,1,9,5


In [28]:
wrong_answers[0:10].drop(['id', 'predictions', 'labels'], axis=1).apply(lambda h: hand_encode(np.array(h)), axis=1)

4202     10♠, J♦, Q♥, K♣, A♠
6823     10♥, A♣, J♣, K♥, Q♠
18740    J♥, A♥, K♣, 10♣, Q♦
20512    J♦, Q♠, K♥, A♣, 10♥
24421     4♥, 2♥, 3♠, A♠, 5♥
28433     5♠, 7♣, 4♣, 8♣, 6♣
29282    Q♠, A♥, K♠, J♠, 10♣
36911    K♦, J♣, A♣, 10♦, Q♥
41314    K♥, Q♥, A♥, 10♣, J♣
46211    10♣, 8♣, K♣, 9♣, A♣
dtype: object